In [27]:
import cv2
import numpy as np

imagename='orangetree.jpg'
testdir='test3/'
# mouse callback function
def draw_circle(event,x,y,flags,param):
    global needdraw
    if event == cv2.EVENT_LBUTTONDOWN:
        needdraw = True
    if event == cv2.EVENT_LBUTTONUP:
        needdraw = False
    if event == cv2.EVENT_MOUSEMOVE:
        if needdraw:
            cv2.rectangle(img,(x,y),(x+8,y+8),(0,0,0),-1)

img = cv2.imread(imagename)
cv2.namedWindow('image')
needdraw=False
cv2.setMouseCallback('image',draw_circle)

while(1):
    cv2.imshow('image',img)
    if cv2.waitKey(20) & 0xFF == 27:
        break
cv2.destroyAllWindows()

origin = cv2.imread(imagename)
originnp = np.array(origin,dtype=int)
after = np.array(img,dtype=int)

mask = originnp - after

for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if not np.array_equal(mask[i][j],[0,0,0]):
            mask[i][j] = [255,255,255]
mask = mask.astype(np.uint8)

cv2.imwrite(testdir+imagename+'_after.jpeg', img)
cv2.imwrite(testdir+imagename+'_mask.jpeg',mask)

mask = mask[:,:,0]
tesla_inpaint = cv2.inpaint(img,mask,3,cv2.INPAINT_TELEA)
ns_inpaint = cv2.inpaint(img,mask,3,cv2.INPAINT_NS)

cv2.imwrite(testdir+imagename+'_tesla_inpaint.jpeg', tesla_inpaint)
cv2.imwrite(testdir+imagename+'_ns_inpaint.jpeg', ns_inpaint)

True

## Other method
### Intro
上面是用Opencv python library 所實做出來的兩種image inpaint的方法的code  
允許使用者用滑鼠在圖片上做mask 再透過inpaint API 查看效果  
兩種方法分別是  
  
 “An Image Inpainting Technique Based on the Fast Marching Method” by Alexandru Telea in 2004  
 演算法大略是參考要被inpaint的周遭pixel,  
 依照靠近程度給予weight,並用fast Marching Method來找下一個要inpaint的pixel  
   
 “Navier-Stokes, Fluid Dynamics, and Image and Video Inpainting” by Bertalmio, Marcelo, Andrea L. Bertozzi, and Guillermo Sapiro in 2001  
 這個方法則是基於流體力學,並利用偏微分方程,先遍歷位置區塊的邊緣
 並持續進行isophotes
   
以下用這5張圖來做三種不同的實驗  
<img src="apple.jpg" title="apple" width="200" height="200" style="float: left" />
<img src="appletree.jpg" title="apple" width="200" height="200" style="float: left" />
<img src="face.jpg" title="apple" width="200" height="200" style="float: left" />
<img src="orange.jpg" title="apple" width="200" height="200" style="float: left" />
<img src="orangetree.jpg" title="apple" width="200" height="200" style="float: left" />

### test1 random thin line
由下面幾張圖可以發現不論使哪種圖片或是方法  
inpaint過後的圖片質量皆不差  
<img src="test1/apple.jpg_ns_inpaint.jpeg" title="apple" width="200" height="200" style="float: left" />
<img src="test1/apple.jpg_tesla_inpaint.jpeg" title="apple" width="200" height="200" style="float: left" />
<img src="test1/orangetree.jpg_ns_inpaint.jpeg" title="apple" width="200" height="200" style="float: left" />
<img src="test1/orangetree.jpg_tesla_inpaint.jpeg" title="apple" width="200" height="200" style="float: left" />
<img src="test1/face.jpg_tesla_inpaint.jpeg" title="apple" width="200" height="200" style="float: left" />


### test2 random thick line
由於未知的範圍增加了,  
相比於test1,圖片就會有明顯類似馬賽克的defect或是mask波紋    
如face圖片中的眼睛就不見了,  
但還是可以看出原本圖片的樣貌,若是周圍背景顏色較為單一,  
也不會與原圖相差太多

### test3 erase object
先前有看過nvidia的[影片](https://www.youtube.com/watch?v=tU484zM3pDY)  
因此想做類似的實驗  
不出意料地,大部分的結果並不是很好  
只有在被刪除的周圍顏色背景差不多時  
才會有比較真實的結果  
但如果是小部分不同的像素想要移除,這依然是個不錯的方法